## Установка пакетов:

`pip install gym[atari]` -- непосредственно наша тестовая среда с различными играми

`pip install tqdm` -- progress bar для python 

`pip install keras` -- библиотека глубинного обучения

In [1]:
%%bash

pip install gym gym[atari] tqdm keras

  Found existing installation: Keras-Applications 1.0.4
    Uninstalling Keras-Applications-1.0.4:
      Successfully uninstalled Keras-Applications-1.0.4
  Found existing installation: Keras-Preprocessing 1.0.2
    Uninstalling Keras-Preprocessing-1.0.2:
      Successfully uninstalled Keras-Preprocessing-1.0.2


In [2]:
import random
import gym

import numpy as np
from collections import deque
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, InputLayer
from keras.optimizers import Adam
from tqdm import tqdm, tqdm_notebook
import cv2
import time
from IPython.display import display, Image

import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [3]:
(keras.__version__, gym.__version__)

('2.2.4', '0.10.9')

## Знакомство с OpenAI Gym

# TODO

[OpenAI Gym](https://gym.openai.com/) -- это фреймворк с коллекцией разнообразных тестовых сред для обучения, наподобие набора данных ImageNet.

Основная идея стоит в стандартизации тестовых сред для более легкого воспроизведения результатов научных публикаций.

За основые среды для обучения наших моделек, а также для последующего соревнования возьмем среды игр Atari:

* [Breakout](https://gym.openai.com/envs/Breakout-v0/)

* [SpaceInvaders](https://gym.openai.com/envs/SpaceInvaders-v0)

* [MsPacman](https://gym.openai.com/envs/MsPacman-v0/)

Посмотрим на одну из игр подробнее. В силу стандартизированности тестовых сред, для изучения других игр вам понадобится изменить только название среды :)

In [4]:
env_name = "Breakout-v0"
env = gym.make(env_name)

`env` -- класс той самой среды, которую мы запускаем

Посмотрим, что мы можем извлекать из этого класса:

### пример запуска тестовой среды от Atari

Запустим код ниже.

Процесс начинается с вызова `env.reset()`, который возвращает начальное наблюдение в игре (в данных играх, наблюдение -- это картинка параметы которой описаны в `env.observation_space`).

`env.render()` запускает окно с отрисовкой текущего наблюдения

Для закрытия окна не забывайте делать `env.close()`

In [5]:
try:
    for i_episode in range(20):
        observation = env.reset()
        print(i_episode)
        for t in range(500):
            time.sleep(1./30)
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                print("Episode finished after {} timesteps".format(t+1))
                break
except KeyboardInterrupt:
    pass
env.close()

0
Episode finished after 259 timesteps
1
Episode finished after 386 timesteps
2
Episode finished after 165 timesteps
3
Episode finished after 185 timesteps
4
Episode finished after 273 timesteps
5
Episode finished after 233 timesteps
6
Episode finished after 246 timesteps
7
Episode finished after 185 timesteps
8
Episode finished after 206 timesteps
9
Episode finished after 278 timesteps
10
Episode finished after 171 timesteps
11
Episode finished after 180 timesteps
12
Episode finished after 170 timesteps
13
Episode finished after 300 timesteps
14
Episode finished after 179 timesteps
15
Episode finished after 190 timesteps
16
Episode finished after 234 timesteps
17
Episode finished after 180 timesteps
18
Episode finished after 236 timesteps
19
Episode finished after 170 timesteps


По сути, это код работы рандомного агента. Его действия -- это элементы пространства действий игры, причем их выбор этих действий равновероятен 

Более подробно про среды и работу с ними вы можете прочитать в [документации на официальном сайте OpenAI Gym](https://gym.openai.com/docs/)

## Инициализация модели

За бейзлайн возьмем алгоритм DQN, выход которого равен количеству действий в играх (для игр Atari равен количеству кнопок на джойстике, а именно 18). 

Для простого старта обучения, вам предоставляется класс DQN-агента (более сложные методы можно найти [тут](https://github.com/keon/deep-q-learning))

Ссылки для более подробного изучения:

* [статья о DQN на towards data science](https://towardsdatascience.com/welcome-to-deep-reinforcement-learning-part-1-dqn-c3cab4d41b6b)

* [фреймворк с RL-моделями на keras](https://github.com/keras-rl/keras-rl)

* [Релизация DQN на pytorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)


In [6]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 0.05  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(InputLayer(input_shape=self.state_size))
        for _ in range(2):
            model.add(Conv2D(8, (3, 3), activation='relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

## Обучение модели

In [7]:
env_name = "Breakout-v0"
env = gym.make(env_name)

GRAYSCALE = True
observation = env.reset()
downsample = 4
new_shape = [i // downsample if i > 3 else i for i in observation.shape]
if GRAYSCALE:
    new_shape[-1] = 1
new_shape = tuple(new_shape)

action_size = env.action_space.n
agent = DQNAgent(new_shape, action_size)
##agent.load("./pong_2.h5")
done = False
batch_size = 32

def process_state(state, grayscale=GRAYSCALE):
    if grayscale:
        state = cv2.cvtColor(state, cv2.COLOR_BGR2GRAY)
    state = cv2.resize(state, new_shape[1::-1])
    if grayscale:
        state = np.reshape(state, (1,) + state.shape + (1,)) / 255.
    else:
        state = np.reshape(state, (1,) + state.shape) / 255.
    return state

EPISODES = 10000

for e in range(EPISODES):
    state = env.reset()
    state = process_state(state)
    total_reward = 0
    for time in tqdm_notebook(range(1000)):
        env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        next_state = process_state(next_state)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, time: {}, e: {:.2}"
                  .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
        
    print("epoch {}, total_reward = {}".format(e, total_reward))
    # if e % 10 == 0:
    #     agent.save("./save/cartpole-dqn.h5")
env.close()

episode: 0/10000, time: 167, e: 0.025
epoch 0, total_reward = 0.0


episode: 1/10000, time: 211, e: 0.01
epoch 1, total_reward = 0.0


episode: 2/10000, time: 346, e: 0.01
epoch 2, total_reward = 2.0


episode: 3/10000, time: 401, e: 0.01
epoch 3, total_reward = 4.0


episode: 4/10000, time: 305, e: 0.01
epoch 4, total_reward = 2.0


...

episode: 853/10000, time: 204, e: 0.01
epoch 853, total_reward = 1.0


episode: 854/10000, time: 361, e: 0.01
epoch 854, total_reward = 3.0


episode: 855/10000, time: 334, e: 0.01
epoch 855, total_reward = 2.0


episode: 856/10000, time: 236, e: 0.01
epoch 856, total_reward = 1.0


KeyboardInterrupt: 

In [ ]:
agent.save("./Breakout-v0_1.h5")

In [15]:
env.step()

TypeError: step() missing 1 required positional argument: 'action'

In [17]:
next_state, reward, done, info = env.step(action)
next_state

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [22]:
action

0

## Тестирование моделей (компьютер vs модель)

In [0]:
state = env.reset()
state = cv2.resize(cv2.cvtColor(state, cv2.COLOR_RGB2GRAY), None, fx=0.5, fy=0.5)
state = np.reshape(state, [1, state_size])
score = 0

for time in range(1000):
    env.render()
    action = agent.act(state)
    next_state, reward, _, _ = env.step(action)
    score += reward
    next_state = cv2.resize(cv2.cvtColor(next_state, cv2.COLOR_RGB2GRAY), None, fx=0.5, fy=0.5)
    state = np.reshape(next_state, [1, state_size])
env.close()
print("You score: {}".format(score))